# Climate Observations: Data Analysis and Visualization

*[Lea Miko Versbach](https://www.maths.lu.se/staff/lea-miko-versbach/), PhD student Lund University, Numerical analysis*  

### Outline:
[1. Introduction](#Introduction)   
&ensp;[1.1 Prerequisites](#Prerequisites)  
[2. General Information](#GeneralInformation)  
[3. Data Processing](#DataProcessing)  
[4. Data Visualization](#DataVisualization)  
[5. Statistics of Climate Observations](#Statistics)    
&ensp;[5.1 Standard Error of Means](#StandardError)  
&ensp;[5.2 Climate Zone](#ClimateZone)  
&ensp;[5.3 Temperature Trends: Poylonomial Regression](#TemperatureTrends)  
&ensp;[5.4 Moving Averages](#MovingAverages)  
&ensp;[5.5 Oscillations](#Oscillations)  
&ensp;[5.6 Correlation](#Correlation)  
[6. Conclusions](#Conclusions)  
[References](#References)

<a id = 'Introduction'></a>
## 1. Introduction

In this project climate data for a city in southern Germany is read in, cleaned and analyzed using different statistical methods. I chose to analyze climate data of the city Regensburg since I lived there most of my childhood.    
My research does not include working with big data sets, therefore this project has been used to learn more about data science, i.e. working with data sets, presenting and analyzing them. Moreover, some literature search was done on how to present climate data and apply some statistical methods on it.  
The project work included searching for freely available climate data sets, reading in the data and getting a first overview, followed by some literature search on how to work with and interpret climate data.    
In the first section of this notebook a bokeh map is created with some basic facts about the station the climate data comes from. In the next section, the data is processed, i.e. read in, cleaned, reorganized and stored to an excel file. In the third section, the user can get a visual overview over the data by using two interactive plots where the available observations can be visualized using different options like mean over all years, yearly data and more. In the following section, some statistical methods are applied to the data sets in order to interpret them. Finally, some conclusions are made in the last section.

<a id = 'Prerequisites'></a>
### 1.1 Prerequisites

Those are the main packages used in this code and therefore are imported in the beginning:

In [ ]:
import numpy as np #scientific computing
import pandas as pd #data handling
import matplotlib.pyplot as plt #plotting
%matplotlib inline 
import geopandas as gpd #plotting
from bokeh.plotting import figure #plotting
from bokeh.io import show, output_notebook #plotting
from bokeh.palettes import Category10 #plotting
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool, Title #plotting
from bokeh.transform import factor_cmap #plotting
from bokeh.models.widgets import Slider #plotting
from bokeh.layouts import column, row, WidgetBox #plotting
from ipywidgets import interact, interactive, fixed, widgets #plotting

<a id='GeneralInformation'></a>
## 2. General Information

In this project, climate data from the years 1879-2019 for the city Regensburg, located in the south-east of Germany, is analyzed. Regensburg is located in eastern Bavaria, at the northest points of the Danubea and the fourth-largest city in the State of Bavaria with more than 150,000 inhabitants.  

<img src="Figures/Regensburg.png" width="500" align="left"/>

The following map of Europe shows where Regensburg is located and provides some basic information about the city. It is created using `bokeh` and `geopandas`:

In [ ]:
#manually set up data frame (information from Wikipedia, date: 10-04-2021)
RGBGdata = pd.DataFrame({'country': 'Germany', 'state': 'Bavaria', 'station': 'Regensburg', 'area': [8076], 'population': [153094],
                       'lat': [49.1], 'lon': [12.5]})

source = ColumnDataSource( data = RGBGdata ) #data source

location = {'RGBG': {'lat':49.1,'lon':12.5}} #location on the map

#load map from geopandas
europe = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
geo_source = GeoJSONDataSource(geojson=europe.to_json())

#get map with tools on the left (they are self-explaining)
p = figure(tools = "pan, wheel_zoom, reset", x_range = (-30, 60), y_range = (30, 85), 
           x_axis_location=None, y_axis_location=None, plot_width=800, plot_height=700)

p.title.text = 'Map of Europe with Regensburg' #title of figure
p.title.text_font_size = '14pt' #font size title

p.add_layout(Title(text='Data taken from Wikipedia https://en.wikipedia.org/wiki/Regensburg', align='left',
       text_font_size="10px", text_color='blue'), "below") #extra information below figure

#remove grid on map
p.xgrid.visible = False
p.ygrid.visible = False

#style of patches
p.patches('xs', 'ys', fill_alpha=0,
    line_color=Category10[10][7], line_width=1.5, source=geo_source)

#style of plot
glyph = p.scatter('lon','lat', source = source, 
              size = 5, fill_alpha = .5, hover_fill_alpha = 1.0,
              marker = 'circle',
              legend_label = 'Regensburg')

#widgets on RHS
glyph_hover = HoverTool(renderers=[glyph],
                         tooltips=[('Sation','@station'),('Longitude','@lon E'), ('Latitude','@lat N'),
                                  ('Country','@country'), ('State','@state'), ('Population','@population'),
                                  ('Area','@area')])
p.add_tools(glyph_hover) #add the widgets

#position title and legend
p.title.align = 'center'
p.legend.location = 'top_left'

#show figure in notebook
output_notebook()
show(p)

<a id='DataProcessing'></a>
## 3. Data Processing

The data is downoladed from the homepage of the DWD (Deutscher Wetterdienst), the German Meteorological Service
and can be found at [this url](https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivtagmonat.html?nn=16102). They provide free historical weather data for 83 weather stations in Germany. The download consists of several html and txt files, which provide all data as well as explanations for the given data.

We start by reading in the data, cleaning it and making it more readable. This is done using the `pandas` package:

In [ ]:
#read in the file using pandas as dataframe
df = pd.read_table('Data/produkt_klima_tag_18790101_20191231_04104.txt',header=0,sep=';|\t',engine='python')

We check how the data actually looks like in the beginning.

In [ ]:
df.head() #print first rows of table

First we clean the data, i.e. delete unnecessary columns or columns that are not of interest for this project using the pandas `drop` function and selecting the columns by their names. Setting the `inplace` parameter to true ensures that the respective column is deleted from the data set.

In [ ]:
df.drop(columns='STATIONS_ID',inplace=True) #delete the unnecessary station ID column, we consider only one station ID
df.drop(columns='QN_3',inplace=True) #delete the column describing measurement quality
df.drop(columns='QN_4',inplace=True) #delete the column describing measurement quality
df.drop(columns='RSKF',inplace=True) #delete the column describing numerical code for precipitation
df.drop(columns='eor',inplace=True) #delete the unnecessary eor (end of row) column
#df.head()

We change the names in the table since they are not clear. A description of the abbreviations can be found (in German) in the html file.

In [ ]:
df.columns = ['date','max wind peak value','average wind velocity','precipitation level','sun hours',
              'snow height','average degree of coverage','average degree of vaport pressure',
              'average air pressure','average temperature','average relative humidity',
              'max temperature','min temperature','min temperature 5cm'] #rename remaining columns
#df.head()

Next, we change the index of the columns. We replace the numerical index by the date information, using pandas in-built datetime form `to_datetime` and create a multiindex for year-month-day.

In [ ]:
df.set_index(['date'],inplace=True) #set DataFrame index using column 'date'
df.index = pd.to_datetime(df.index, format='%Y%m%d') #use pandas datetime option to replace by year-month-day format
df['day'] = df.index.day #define day index
df['month'] = df.index.month #define month index
df['year'] = df.index.year #define year index
df.set_index(['year','month','day'],inplace=True) #set DataFrame index using multicolumns 'year', 'month', 'day'
#df.head()

Missing data is denoted by -999 in the dataset. We replace these values by `numpy` $\texttt{NaN}$ (not a number) using the pandas `replace` function.

In [ ]:
df = df.replace([-999],np.nan) #replace all non-given values (-999) by numpy NaN

This was the final step in the data preparation. We check how the data set is presented now and save the processed data to the disk.

In [ ]:
df.head() #check how the data looks now

It remains to save the processed data as an excel list to the disk into the *Data* folder. 

In [ ]:
output = pd.ExcelWriter('Data/ClimateRgbg.xlsx')
df.to_excel(output,'Sheet1')
output.save()

<a id='DataVisualization'></a>
## 4. Data Visualization

To get an overwiew over the existing data, in this section the user can look at the yearly averaged data for each climate observation. As averaging method one can choose min, max, giving the minimal and maximal measured value of the chosen observation, respectively, or mean, calculating the mean over one calendar year.

Fist, we define global settings for all figures in this notebook:

In [ ]:
plt.rcParams.update({ 'figure.figsize': [7.2,4.5], 'font.size': 14,'figure.titlesize': 'large',#[6.4, 4.8][3.5, 1.5]
                      'figure.titleweight': 'bold',
                      'axes.titlepad': 12.0,})
plt.rcParams.update({ 'axes.linewidth':1.2, 'axes.labelpad':10, 
                     'axes.labelsize': 14, 'axes.labelweight': 'bold' })
plt.rcParams.update({ 'xtick.labelsize':12, 'ytick.labelsize':12 })
plt.rcParams.update({ 'xtick.major.width': 1.2, 'ytick.major.width': 1.2, 
                    'xtick.major.size': 8, 'ytick.major.size': 8 })

The plots are produced using interactive `bokeh` plots and pandas `groupby`, where we choose a specific column from the dataframe (specified by the chosed observation), group by year and then apply `agg` to either get the minimal, maximal or the average value per year. The climate observation can be chosen via a dropdown box and the yearly averaging method via radio buttons.

In [ ]:
def YearlyAveragePlot(observation,option):

    if observation=='temp':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='temperature (°C)',
          title='Yearly {} temperature'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['min temperature','average temperature','max temperature'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option]) #get the DataFrame
            ddf.squeeze() #squeeze 1 dimensional axis object into scalar
            x,y = ddf.index.values, ddf.values #define x and y
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs) #plotting settings
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)
            
    if observation=='wind':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='velocity (m/sec)',
          title='Yearly {} wind'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['max wind peak value','average wind velocity'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)
            
    if observation=='prec':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='precipitation level (mm)',
          title='Yearly {} precipitation level'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['precipitation level'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)
            
    if observation=='snow':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='height (cm)',
          title='Yearly {} snow height'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['snow height'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)
            
    if observation=='sun':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='daily sun (hours)',
          title='Yearly {} sun hours'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['sun hours'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)
            
    if observation=='coverage':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='degree of coverage (Achtel)',
          title='Yearly {} average degree of coverage'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['average degree of coverage'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)
            
    if observation=='pressure':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='pressure (hpa)',
          title='Yearly {} average pressure'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['average degree of vaport pressure','average air pressure'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)

    if observation=='humidity':
        #figure settings (size, label, title, tools)
        p = figure(plot_width=600, plot_height=300,
          x_axis_label='Year', y_axis_label='$\%$',
          title='Yearly {} average relative humidity'.format(option), tools='pan,box_zoom,wheel_zoom,hover,reset',
          tooltips=[('Year','@x'), ('Average {} value year'.format(option),'@y')])

        for obs,color in zip(['average relative humidity'],Category10[10]):
            ddf = df[obs].groupby(['year']).agg([option])
            ddf.squeeze()
            x,y = ddf.index.values, ddf.values
            p.circle(x, y[:,0], radius=.5, color=color, legend_label=obs)
            p.line(x, y[:,0], line_width=1, color=color, legend_label=obs)

    p.add_layout(p.legend[0], 'right') #location legend
    p.title.align = 'center' #location title
    p.title.text_font_size = '14pt' #font size title

    p.xaxis.axis_label_text_font_style = 'normal' #font size label text
    p.yaxis.axis_label_text_font_style = 'normal'

    #show output in the notebook
    output_notebook() 
    show(p)

#create interactive plot using dopdown box and radio button
interactive_plot = interactive(YearlyAveragePlot,observation={'temperature':'temp','wind':'wind','precipitation':'prec',
                                              'snow':'snow','sun':'sun','coverage':'coverage',
                                              'pressure':'pressure','humidity':'humidity'},
                               option=widgets.RadioButtons(options=['max','mean','min'],disabled=False))
     
#plot the figure and widgets
interactive_plot.children[0].description=r'Observation:' #widget interactive_plot
interactive_plot.children[1].description=r'measure:' #widget interactive_plot
display(interactive_plot)

The next plot allows to choose a specific year and observation. The values for each day are plotted, as well as the mean over all years from the data set. For the latter one, again `groupby` is used, this time grouping by both month and day, then choosing the mean for `agg`. The observation of interest can be chosen via a dropdown box and the year via a slider.

In [ ]:
def YearlyPlot(year,observation):

    tempData = df[observation] #get data frame with just average temperature
    tempData.dropna(inplace=True) #drop all NaN years
    
    #check if data for the chosen year exists
    if (year in tempData.index) == False:
        print('No data available for the year {}!'.format(year))
    
    else:
    
        #plot the daily mean observation over all years
        tempData.groupby(['month','day']).agg(['mean']).plot(style='rx',label='mean over all years')
        
        #plot the daily observation for chosen year
        tempData.loc[year].plot(style='bx',legend='{}'.format(year))
        plt.xticks(np.arange(15,360,30), ('January','February','March','April','May','June','July','August','September',
                                'October','November','December'), rotation=45) #manually set xticks
    
        #ylabel for each observation
        if (observation=='max temperature' or observation=='min temperature' or observation=='average temperature'):
            plt.ylabel('temperature (˚C)')
        if (observation=='max wind peak value' or observation=='average wind velocity'):
            plt.ylabel('velocity (m/sec)')
        if (observation=='precipitation level'):
            plt.ylabel('precipitation (mm)')
        if (observation=='snow height'):
            plt.ylabel('height (cm)')
        if (observation=='sun hours'):
            plt.ylabel('hours (daily)')
        if (observation=='average degree of coverage'):
            plt.ylabel('coverage (Achtel)')
        if (observation=='average degree of vaport pressure' or observation=='average air pressure'):
            plt.ylabel('pressure (hpa)')
        if (observation=='average relative humidity'):
            plt.ylabel('$\%$')
            
        plt.xlabel('day,month') #xlabel
        plt.title('{}: {}'.format(year,observation)) #title
        plt.legend() #legend
        
        plt.tight_layout() #tight layout (needed for safing)
        plt.savefig('Figures/YearlyTemp.png',dpi=300) #safe figure
    
#create interactive plot with slider and dopdown box
interactive_plot = interactive(YearlyPlot,
                               year=widgets.IntSlider(min=1879, max=2019, step=1, value=1990,continuous_update=False),
                               observation={'max temperature':'max temperature',
                                           'min temperature':'min temperature',
                                           'average temperature':'average temperature',
                                           'max wind peak value':'max wind peak value',
                                           'average wind velocity':'average wind velocity',
                                           'precipitation level':'precipitation level',
                                           'snow height':'snow height',
                                           'sun hours':'sun hours',
                                           'average degree of coverage':'average degree of coverage',
                                           'average degree of vaport pressure':'average degree of vaport pressure',
                                           'average air pressure':'average air pressure',
                                           'average relative humidity':'average relative humidity'})

#visualization in notebook
interactive_plot.children[0].description=r'Year:' #widget interactive_plot
interactive_plot.children[1].description=r'Observation:' #widget interactive_plot
display(interactive_plot)

<a id='Statistics'></a>
## 5. Statistics of Climate Observations

In the following, we apply some statistical methods to interpret the climate data. Most of the theory is taken from the book *Statistical Analysis of Climate Series -
Analyzing, Plotting, Modeling, and Predicting with R* by Helmut Pruscha (Springer 2013), which is available as ebook in the LU library, see [here](https://eds.b.ebscohost.com/eds/detail/detail?vid=2&sid=68a4b26f-fdd9-4747-95f2-40e5d7637ebc%40sessionmgr102&bdata=JnNpdGU9ZWRzLWxpdmUmc2NvcGU9c2l0ZQ%3d%3d#AN=lub.5961074&db=cat07147a). The interested reader is referred to this reference for more explanation of the used methods.

<a id='StandardError'></a>
### 5.1 Standard Error of Means

Above and in the following we often consider the mean over time intervals as years or months. We are therefore intersted in the standard error of the mean, which is calculated as the standard deviation $\sigma$ devided by the square root of the lenght of the sample $n$:
$$
SEM = \frac{\sigma}{\sqrt{n}}
$$

In [ ]:
#function to compute standar error of the mean
def f_sem(x):
    return x.std() / np.sqrt(x.count())
f_sem.__name__ = 'SEM'

Below we plot the standard error of the mean for each year arond the yearly mean for the chosen observation. As before, `groupy` and `agg` are used as well as the function to compute the SEM. The observation of interest can be chosen via a dropdown box.

In [ ]:
def SEM(option):
    y = df.groupby('year').agg(['mean',f_sem])[option,'mean'] #calculate yearly mean of chosen observation
    plt.plot(y,label='yearly mean') #plot
    yerr = df.groupby('year').agg(['mean',f_sem])[option,'SEM'] #calculate yearly SEM of chosen observation
    plt.fill_between(y.index, (y+yerr).values, (y-yerr).values, alpha=.5,label='SEM') #plot
    
    #set ylabels for chosen observation
    if (option=='max temperature' or option=='min temperature' or option=='average temperature'):
        plt.ylabel('temperature (˚C)')
    if (option=='max wind peak value' or option=='average wind velocity'):
        plt.ylabel('velocity (m/sec)')
    if (option=='precipitation level'):
        plt.ylabel('precipitation (mm)')
    if (option=='snow height'):
        plt.ylabel('height (cm)')
    if (option=='sun hours'):
        plt.ylabel('hours (daily)')
    if (option=='average degree of coverage'):
        plt.ylabel('coverage (Achtel)')
    if (option=='average degree of vaport pressure' or option=='average air pressure'):
        plt.ylabel('pressure (hpa)')
    if (option=='average relative humidity'):
        plt.ylabel('$\%$')
      
    #legend, xlabel and title
    plt.legend()
    plt.xlabel('Year')
    plt.title('Yearly mean {}'.format(option))
    
    #safe figure in tight layout
    plt.tight_layout()
    plt.savefig('Figures/TempYearlyMean.png',dpi=300)
  
#create slider interactive plot
interactive_plot = interactive(SEM,option={'max temperature':'max temperature',
                                           'min temperature':'min temperature',
                                           'average temperature':'average temperature',
                                           'max wind peak value':'max wind peak value',
                                           'average wind velocity':'average wind velocity',
                                           'precipitation level':'precipitation level',
                                           'snow height':'snow height',
                                           'sun hours':'sun hours',
                                           'average degree of coverage':'average degree of coverage',
                                           'average degree of vaport pressure':'average degree of vaport pressure',
                                           'average air pressure':'average air pressure',
                                           'average relative humidity':'average relative humidity'})

#visualization plot
interactive_plot.children[0].description=r'Observation:' # slider interactive_plot
display(interactive_plot)

<a id='ClimateZone'></a>
### 5.2 Climate Zone

To determine the climate zone of Regensburg, we use the [Köppen climate classification](https://en.wikipedia.org/wiki/Köppen_climate_classification), which is one of the most commonly used climate classification systems. It was published by the German-Russian climatologist Wladimir Köppen in the year 1884.  
This classification system is based on interpreting the monthly average values for the average temperature as well as the monthly average precipitation level. This is done by calculating the monthly mean over all years, respectively, and plotting them.  
The following plots show the monthly average values for the average temperature as well as the average precipitation level. Moreover, the user can choose a year of interest to compare the monthly mean average temperature for a specific year to the respective mean over all years.

In [ ]:
def MonthlyTempPlots(year):

    tempData = df['average temperature'] #get data frame with just average temperature
    tempData.dropna(inplace=True) #drop all NaN years
    
    rainData = df['precipitation level'] #get data frame with just precipitation level
    rainData.dropna(inplace=True) #drop all NaN years
    
    if (year in df.index) == False:
        #error message if no data exists for chosen year
        print('No data available for the year {}!'.format(year))
        
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(9, 7)) #create 1x2 subplots
        
        #bar plot using groupy over months and agg mean
        (pd.to_numeric(tempData).groupby(['month']).agg(['mean'])).plot.bar(ax=ax[0]) 
        #fix ticks, labels, title and limits of y-axis
        ax[0].set_xticks(np.arange(12)) 
        ax[0].set_xticklabels(('January','February','March','April','May','June','July','August','September',
                                'October','November','December'), rotation=45)
        ax[0].set_ylabel('temperature (°C)') 
        ax[0].set_xlabel('month')
        ax[0].set_title('Average temperature all years')
        ax[0].set_ylim([-12,25])
        ax[0].get_legend().remove()
        
        #bar plot using groupy over months and agg mean
        (pd.to_numeric(rainData).groupby(['month']).agg(['mean'])).plot.bar(ax=ax[1])
        #fix ticks, labels, title and limits of y-axis
        ax[1].set_xticks(np.arange(12)) 
        ax[1].set_xticklabels(('January','February','March','April','May','June','July','August','September',
                                'October','November','December'), rotation=45)
        ax[1].set_ylabel('precipitation (mm)') 
        ax[1].set_xlabel('month')
        ax[1].set_title('Average precipitation all years')
        ax[1].set_ylim([0,5])
        ax[1].yaxis.set_ticks_position('right')
        ax[1].yaxis.set_label_position("right")
        ax[1].get_legend().remove()
        
        #safe figure with tight layout
        fig.tight_layout()
        fig.savefig('Figures/ClimateZone.png',dpi=300)
        
    else:

        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(9, 7)) #create 2x2 subplots
 
        #bar plot using groupy over months and agg mean
        (pd.to_numeric(tempData).groupby(['month']).agg(['mean'])).plot.bar(ax=ax[0,0])
        #fix ticks, labels, title and limits of y-axis
        ax[0,0].set_xticks(np.arange(12)) 
        ax[0,0].set_xticklabels(['January','February','March','April','May','June','July','August','September',
                                'October','November','December'], rotation=45)
        ax[0,0].set_ylabel('temperature (°C)') 
        ax[0,0].set_xlabel('month')
        ax[0,0].set_title('Average temperature all years')
        ax[0,0].set_ylim([-12,25])
        ax[0,0].get_legend().remove()

        #bar plot using groupy over months and agg mean for chosen year
        (pd.to_numeric(tempData.loc[year]).groupby(['month']).agg(['mean'])).plot.bar(ax=ax[0,1])
        #fix ticks, labels, title and limits of y-axis
        ax[0,1].set_xticks(np.arange(12)) 
        ax[0,1].set_xticklabels(['January','February','March','April','May','June','July','August','September',
                                'October','November','December'], rotation=45)
        ax[0,1].set_ylabel('temperature (°C)')  
        ax[0,1].set_title('Average temperature {}'.format(year))
        ax[0,1].set_ylim([-12,25])
        ax[0,1].yaxis.set_ticks_position('right')
        ax[0,1].yaxis.set_label_position("right")
        ax[0,1].get_legend().remove()

        #bar plot using groupy over months and agg mean 
        (pd.to_numeric(rainData).groupby(['month']).agg(['mean'])).plot.bar(ax=ax[1,0])
        #fix ticks, labels, title and limits of y-axis
        ax[1,0].set_xticks(np.arange(12)) 
        ax[1,0].set_xticklabels(['January','February','March','April','May','June','July','August','September',
                                'October','November','December'], rotation=45)
        ax[1,0].set_ylabel('precipitation (mm)') 
        ax[1,0].set_xlabel('month')
        ax[1,0].set_title('Average precipitation all years')
        ax[1,0].set_ylim([0,5])
        ax[1,0].get_legend().remove()

        #bar plot using groupy over months and agg mean for chosen year
        (pd.to_numeric(rainData.loc[year]).groupby(['month']).agg(['mean'])).plot.bar(ax=ax[1,1])
        #fix ticks, labels, title and limits of y-axis
        ax[1,1].set_xticks(np.arange(12)) 
        ax[1,1].set_xticklabels(['January','February','March','April','May','June','July','August','September',
                                'October','November','December'], rotation=45)
        ax[1,1].set_ylabel('precipitation (mm)')  
        ax[1,1].set_title('Average precipitation {}'.format(year))
        ax[1,1].set_ylim([0,5])
        ax[1,1].yaxis.set_ticks_position('right')
        ax[1,1].yaxis.set_label_position("right")
        ax[1,1].get_legend().remove()
 
        #safe figure with tight layout
        fig.tight_layout()
        fig.savefig('Figures/ClimateZone.png',dpi=300)

#create interactive plot with slider to choose year
interactive_plot = interactive(MonthlyTempPlots,year=widgets.IntSlider(min=1879, max=2019, step=1, value=1990,continuous_update=False))
interactive_plot.children[0].description=r'Year' # slider interactive_plot
display(interactive_plot) #display plot in notebook

Due to the [Köppen climate classification](https://en.wikipedia.org/wiki/Köppen_climate_classification) the climate in Regensburg can be classified between humid continental (Dfb) and oceanic (Cfb) climate.

<a id='TemperatureTrends'></a>
### 5.3 Temperature Trends: Poylonomial Regression

The climate change and the increasing average temperature can also be noticed in this climate data set. In order to see an overall temperature trend, we use polynomial regression. Using these fitted polynomials (of lower order), the long-term variation of the time series can be described mathematically. This allows us to check if we can notice the climate change in the yearly average temperature development between 1879 and 2019.

The following methods need to be imported from the `sklearn` package, which provides tools for predictive data analysis and Machine Learning:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

By changing $\texttt{order}$ in the next code snipplet, the order of the interpolating polynomial can be adapted.

In [ ]:
data = pd.to_numeric(df['average temperature']).groupby(['year']).agg(['mean']) #choose wanted data by groupby
#reshaping needed for sklearn function
x = data.index.values.reshape(-1,1)
y = data.iloc[:,0].values.reshape(-1,1)

order = 4 #order of the polynomial approximation

#create polynomial regression using sklearn functions
poly_reg = PolynomialFeatures(degree=order)
X_poly = poly_reg.fit_transform(x)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y)

plt.plot(x, y, label='yearly average') #plot yearly mean temperature
plt.plot(x, pol_reg.predict(poly_reg.fit_transform(x)), label='fitted polynomial') #plot fitted polynomial
#set xticks, title, labels, legend
plt.xticks([1880,1900,1920,1940,1960,1980,2000,2019], [1897,1900,1920,1940,1960,1980,2000,2019]) 
plt.title('Temperature trend using polynomial regression of order {}'.format(order))
plt.xlabel('year')
plt.ylabel('average temperature (°C)')
plt.legend()
plt.savefig('Figures/TempTrend.png',dpi=300) #safe figure

From the previous figure it is obvious that the temperature average increased between 1879 and 2019.

We can calculate the residual between the fitted polynomial $p_t$ and the actual time series $y_t$ for each year $t$, which allows to measure the goodness-of-fit using the mean sum of the squared residuals (MSQ):
$$
\sqrt{\frac{1}{n}\sum_{t=1}^{n}(y_t-p_t)^2}.
$$
A smaller value indicates a better polynomial approximation.
For the given approximation we can calculate the *goodness-of-fit measure*:

In [ ]:
p = pol_reg.predict(poly_reg.fit_transform(x)) #fitted polynomial (array)
L = (y-p)**2 #subtract arrays and square each entry
L2 = [(1/(i+1))*L[i] for i in range(len(L))] #list comprehension for multiplication factor
print('goodness-of-fit measure:',np.sqrt(sum(L2))) #sum over list and print answer

Changing the order above and re-running the previous two cells shows the dependence of the goodness of the polynomial approximation on the order of the polynomial.

<a id='MovingAverages'></a>
### 5.4 Moving Averages

Another interesting insight into the temperature development over the years is to compare several consecutives years by calculating yearly averages. This is done by computing so-called *moving averages* over $k$ years with $k=2l+1$. We can calculate a trend temperature $m_t$ in a time interval $[t-l,t+t]$ as an average over $k$ years centered at $t$ and weighting the endpoints by $\frac{1}{2}$:
$$
m_t = \frac{1}{2l}\left(\frac{1}{2}y_{t-l}+y_{t-l+1}+\dots+y_t+\dots+y_{t+l-1}+\frac{1}{2}y_{t+l}\right).
$$
The following function computes for a time series $\texttt{x}$ the moving averages over $\texttt{k}$ years:

In [ ]:
def ma(x,k):
    #computes the trend function for a time series x for k years, k needs to be odd!
    l = int((k-1)/2)
    y = np.zeros(len(x)-2*l)
    for i in range(len(x)-2*l):
        t = i+l
        y[i] = sum(x[t-l+1:t+l]) + 0.5*(x[t-l]+x[t+l])
    return 1/(2*l)*y

Then we can plot the moving averages over $k$ years, where $k$ can be chosen by the user using a slider.

In [ ]:
def MovingAveragesPlot(k):
    
    z = ma(y,k) #calculate moving averages over k year
    l = int((k-1)/2)
    
    #plot, set xticks, title, labels and save figure
    plt.plot(x[l:-l],z,label='centered moving averages')
    plt.plot(x, y, label='yearly average')
    plt.xticks([1880,1900,1920,1940,1960,1980,2000,2019], [1897,1900,1920,1940,1960,1980,2000,2019])
    plt.title('Moving averages with means of {} years'.format(k))
    plt.xlabel('year')
    plt.ylabel('average temperature (°C)')
    plt.legend()
    plt.savefig('Figures/TempMovingAverages.png',dpi=300)

#create interactive plot with slider to choose number of years to average over
interactive_plot = interactive(MovingAveragesPlot,k=widgets.IntSlider(min=3, max=11, step=2, value=1,continuous_update=False))
interactive_plot.children[0].description=r'Years:' # slider interactive_plot
#display interactive plot
display(interactive_plot)

<a id='Oscillations'></a>
### 5.5 Oscillations

Not only trends and moving averages are interesting, but also the oscillation of climatological series over the years. These are calculated considering moving 10-year blocks $[t-9,t]$ and calcualting the standard deviation for each block.  
This is done in the following function `Oscillation`:

In [ ]:
def Oscillation(x,k):
    #calculate standard deviation of series x over moving k-year blocks
    y = np.zeros(len(x)-k)
    for i in range(len(x)-k):
        t = i+k
        y[i] = np.std(x[t-k:t+1])
    return y

In the next plot, we can see how much the average temperature oscillates.

In [ ]:
plt.plot(x[10:], Oscillation(y[:,0],10)) #plot std oscillations against years
#set xticks, label, title and save figure
plt.xticks([1880,1900,1920,1940,1960,1980,2000,2019], [1897,1900,1920,1940,1960,1980,2000,2019])
plt.xlabel('year')
plt.ylabel('standard deviation temperature (°C)')
plt.title('Oscillation yearly average temperature')
plt.savefig('Figures/TemperatureOscillation.png',dpi=300)

<a id='Correlation'></a>
### 5.6 Correlation

Looking at the correlating of data reveals more information about the dependence between the datasets. 
The correlation coefficient is a statistical measure of the strength of the dependence between the datasets with values ranging between -1.0 and 1.0.  
Auto-correlation is a special case where the dependence between variables from the same dataset at different time points is studied. This is an indication for how the value of a parameter $y_t$ at time point $t$ is correlated with the value $y_{t+1}$ at time point $t+1$. In the case of climate data, auto-correlation can indicate a predictive relationship.  
Here, we study the effect of seasonal and trend components on auto-correlation. We therefore make a scatter plot of $y_t$ against $y_{t+1}$ and calculate the correlation coefficient using the numpy function `corrcoef`.

In [ ]:
#monthly mean temperature using groupy and agg
mt = pd.to_numeric(df['average temperature']).groupby(['year','month']).agg(['mean'])['mean'] 

corr = np.corrcoef(mt[:-1],mt[1:])[0,1] #calculate correlation coefficient
#create scatter plot, labels, legend, title and save figure
plt.scatter(mt[:-1],mt[1:],label='correlation coefficient: {:.3f}'.format(corr))
plt.xlabel('$\mathbf{y_t}$ (°C)')
plt.ylabel('$\mathbf{y_{t+1}}$ (°C)')
plt.legend()
plt.title('Correlation monthly temperature average')
plt.savefig('Figures/TempCorrel.png',dpi=300)

We notice that the correlation coefficient is $r \approx 0.825$. The large value is caused by seasonal effects, i.e. to the course of the monthly temperatures over the calendar year and the dependence of the monthly temperature on the previous month. It thus contains redundant information. To overcome this effect, we can adjust the temperatures seasonally by subtracting the monthly average temperatures over all considered years. This can be done by subtracting the corresponding data frames.

In [ ]:
tempData = df['average temperature'] #get data frame with just average temperature
tempData.dropna(inplace=True) 
av = pd.to_numeric(tempData).groupby(['month']).agg(['mean']) #get monthly mean over all years
d1 = pd.DataFrame(mt) #create DataFrame from monthly mean for each year
rr = d1-av #subtract these data frames to seasonally adjust

corr = np.corrcoef(rr['mean'][:-1],rr['mean'][1:])[0,1] #calculate seasonally adjusted correlation coefficient
#create scatter plot, labels, legend, title and save figure
plt.scatter(rr[:-1],rr[1:],label='correlation coefficient: {:.3f}'.format(corr))
plt.xlabel('$\mathbf{y_t}$ (°C)')
plt.ylabel('$\mathbf{y_{t+1}}$ (°C)')
plt.legend()
plt.title('Correlation monthly temperature average, seasonally adjusted')
plt.savefig('Figures/TempCorrelSeasonallyAdj.png',dpi=300)

We notice that the correlation coefficient decreases to $r \approx 0.247$ when removing seasonal effects.

<a id='Conclusions'></a>
## 6. Conclusions

In this project we have made a small data science case study: We started searching for a freely available dataset and downloaded it. Then we read in the data to a Phyton file and processed the data by cleaning it and improving its readability. In order to get a first overview, we visualized the data using interactive bokeh plots. In the following, we applied some statistical methods to determine the climate zone, climate trends over the considered time intervals as well as correlations between the measurements.  

<!--bibtex

@book{pruscha2012statistical,
  title={Statistical analysis of climate series: analyzing, plotting, modeling, and predicting with R},
  author={Pruscha, Helmut},
  year={2012},
  publisher={Springer Science \& Business Media}
}

-->

<a id='References'></a>
## References

* The data set used in this project is freely available at the webpage of the German Meteorological Service (Deutscher Wetterdienst, DWD), see 
[this url](https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivtagmonat.html?)
* The theory for the statistical methods is mostly taken from  the book *Statistical Analysis of Climate Series - Analyzing, Plotting, Modeling, and Predicting with R* by Helmut Pruscha (Springer 2013)
* Course literature of the course *Reproducible and Interactive Data Science* at Lund University, 2021, see [the course homepage](https://github.com/mlund/jupyter-course)
* Wikipedia pages as linked in the notebook

Rules for using DWD data sets can be found [here](https://www.dwd.de/EN/service/copyright/templates_dwd_as_source.html).